# Set up Milvus Vector DB

### Import necessary utilities

##### Libraries

In [1]:
import os

from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_openai import OpenAIEmbeddings

from langchain.vectorstores.milvus import Milvus

from dotenv import load_dotenv, find_dotenv

##### .env variables

In [2]:
load_dotenv(find_dotenv())

True

##### Paths

In [5]:
main_path = os.getcwd()
docs_path = os.path.join(main_path, 'docs')

### Load the documents

In [6]:
loader = PyPDFDirectoryLoader(docs_path)
documents = loader.load()

##### Split document data in chunks

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap = 30,
    length_function = len,
)
docs = text_splitter.split_documents(documents=documents)

### Create vector database

In [8]:
vector_db = Milvus.from_documents(
    documents=docs,
    embedding=OpenAIEmbeddings(),
    collection_name="banorte_test",
    connection_args={ "host": "localhost", "port": "19530" },
    drop_old=True
)

### Test vector database retrieval

In [9]:
query="¿Qué tarjeta de credito me sirve si tengo un sueldo mensual de $10,000 pesos?"
result_docs = vector_db.similarity_search(query)

print(f"Question: {query}")
print("Result: ", result_docs[0])

Question: ¿Qué tarjeta de credito me sirve si tengo un sueldo mensual de $10,000 pesos?
Result:  page_content='Aprovecha el nuevo beneficio exclusivo de tu Tarjeta de Crédito POR Ti Banorte y paga cómodamente a 12 Meses con Intereses por compras de $4,000 pesos con una tasa de interés del 23% anual fija (2) • Convierte tus compras grandes en pagos pequeños con un plazo e interés fijo con Plan de Pagos' metadata={'source': '/Users/patriciocantutrevino/Documents/Programs/Projects/Learn_Langchain/BanorteTest/docs/folleto-informativo-por-ti.pdf', 'page': 0, 'pk': 448315427240807449}


### Retrieve created collection

In [ ]:
vector_db_retrieve = Milvus(
    embedding_function=OpenAIEmbeddings(),
    connection_args={ "host": "localhost", "port": "19530" },
    collection_name="banorte_test"
)